In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install pyspark

Mounted at /content/drive
     |████████████████████████████████| 281.4 MB 29 kB/s 
     |████████████████████████████████| 198 kB 48.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=d39e6c94a1b33589d082c470766e6dec935b978c9ce84928a2e7c369878a6a33
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import os
import pandas as pd
import numpy as np

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[2]").appName("bone-marrow").getOrCreate()

spark

In [ ]:
df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/bone-marrow.csv',inferSchema=True,header=True)

df.show(3)

+---+---------------+--------------+---------+----------+----+-----------+--------+------------+-----------+--------+---------+--------+------------+-------+---------+-------------+------------+--------+-----------+-------+----+------+------------+--------------+---------------+-------+----------+--------+-----------+---------+-----------+---------+-----------+-----------+--------------------+-------------+---------------+
| id|Recipientgender|Stemcellsource| Donorage|Donorage35|IIIV|Gendermatch|DonorABO|RecipientABO|RecipientRh|ABOmatch|CMVstatus|DonorCMV|RecipientCMV|Disease|Riskgroup|Txpostrelapse|Diseasegroup|HLAmatch|HLAmismatch|Antigen|Alel|HLAgrI|Recipientage|Recipientage10|Recipientageint|Relapse|aGvHDIIIIV|extcGvHD|CD34kgx10d6| CD3dCD34|CD3dkgx10d8|Rbodymass|ANCrecovery|PLTrecovery|time_to_aGvHD_III_IV|survival_time|survival_status|
+---+---------------+--------------+---------+----------+----+-----------+--------+------------+-----------+--------+---------+--------+----------

In [ ]:
#View the shape structure of the dataset to determine the size of the dataset
print((df.count(),len(df.columns)))

#View the data type of the dataset
df.printSchema()

#The describe function views the statistical indicators of the dataset
df.describe().show(5,False)

# Group by survival_status results and view the distribution of results
result_df = df.groupBy("survival_status").count().sort("survival_status", ascending=False)
result_df.show()

#change ? values to NaN
df = df.na.replace('?', None)


(187, 38)
root
 |-- id: integer (nullable = true)
 |-- Recipientgender: integer (nullable = true)
 |-- Stemcellsource: integer (nullable = true)
 |-- Donorage: double (nullable = true)
 |-- Donorage35: integer (nullable = true)
 |-- IIIV: integer (nullable = true)
 |-- Gendermatch: integer (nullable = true)
 |-- DonorABO: integer (nullable = true)
 |-- RecipientABO: string (nullable = true)
 |-- RecipientRh: string (nullable = true)
 |-- ABOmatch: string (nullable = true)
 |-- CMVstatus: string (nullable = true)
 |-- DonorCMV: string (nullable = true)
 |-- RecipientCMV: string (nullable = true)
 |-- Disease: string (nullable = true)
 |-- Riskgroup: integer (nullable = true)
 |-- Txpostrelapse: integer (nullable = true)
 |-- Diseasegroup: integer (nullable = true)
 |-- HLAmatch: integer (nullable = true)
 |-- HLAmismatch: integer (nullable = true)
 |-- Antigen: string (nullable = true)
 |-- Alel: string (nullable = true)
 |-- HLAgrI: integer (nullable = true)
 |-- Recipientage: double (

In [ ]:
#Change datatype
from pyspark.sql.types import DoubleType
df = df.withColumn("RecipientABO",col("RecipientABO").cast(DoubleType())) \
    .withColumn("RecipientRh",col("RecipientRh").cast(DoubleType())) \
    .withColumn("ABOmatch",col("ABOmatch").cast(DoubleType())) \
    .withColumn("CMVstatus",col("CMVstatus").cast(DoubleType())) \
    .withColumn("DonorCMV",col("DonorCMV").cast(DoubleType())) \
    .withColumn("RecipientCMV",col("RecipientCMV").cast(DoubleType())) \
    .withColumn("Antigen",col("Antigen").cast(DoubleType())) \
    .withColumn("Alel",col("Alel").cast(DoubleType())) \
    .withColumn("extcGvHD",col("extcGvHD").cast(DoubleType())) \
    .withColumn("CD3dCD34",col("CD3dCD34").cast(DoubleType())) \
    .withColumn("CD3dkgx10d8",col("CD3dkgx10d8").cast(DoubleType())) \
    .withColumn("Rbodymass",col("Rbodymass").cast(DoubleType()))
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Recipientgender: integer (nullable = true)
 |-- Stemcellsource: integer (nullable = true)
 |-- Donorage: double (nullable = true)
 |-- Donorage35: integer (nullable = true)
 |-- IIIV: integer (nullable = true)
 |-- Gendermatch: integer (nullable = true)
 |-- DonorABO: integer (nullable = true)
 |-- RecipientABO: double (nullable = true)
 |-- RecipientRh: double (nullable = true)
 |-- ABOmatch: double (nullable = true)
 |-- CMVstatus: double (nullable = true)
 |-- DonorCMV: double (nullable = true)
 |-- RecipientCMV: double (nullable = true)
 |-- Disease: string (nullable = true)
 |-- Riskgroup: integer (nullable = true)
 |-- Txpostrelapse: integer (nullable = true)
 |-- Diseasegroup: integer (nullable = true)
 |-- HLAmatch: integer (nullable = true)
 |-- HLAmismatch: integer (nullable = true)
 |-- Antigen: double (nullable = true)
 |-- Alel: double (nullable = true)
 |-- HLAgrI: integer (nullable = true)
 |-- Recipientage: double (nullable =

In [ ]:
#View the number of null values in each column
df_agg = df.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns])  
df_agg.show()

#Check the data type of the input column again. 
df.printSchema()

#Directly delete rows with missing values
#Process duplicate values
df = df.dropna()
df = df.dropDuplicates()
print((df.count(),len(df.columns)))

+---+---------------+--------------+--------+----------+----+-----------+--------+------------+-----------+--------+---------+--------+------------+-------+---------+-------------+------------+--------+-----------+-------+----+------+------------+--------------+---------------+-------+----------+--------+-----------+--------+-----------+---------+-----------+-----------+--------------------+-------------+---------------+
| id|Recipientgender|Stemcellsource|Donorage|Donorage35|IIIV|Gendermatch|DonorABO|RecipientABO|RecipientRh|ABOmatch|CMVstatus|DonorCMV|RecipientCMV|Disease|Riskgroup|Txpostrelapse|Diseasegroup|HLAmatch|HLAmismatch|Antigen|Alel|HLAgrI|Recipientage|Recipientage10|Recipientageint|Relapse|aGvHDIIIIV|extcGvHD|CD34kgx10d6|CD3dCD34|CD3dkgx10d8|Rbodymass|ANCrecovery|PLTrecovery|time_to_aGvHD_III_IV|survival_time|survival_status|
+---+---------------+--------------+--------+----------+----+-----------+--------+------------+-----------+--------+---------+--------+------------+--

In [ ]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

#View column names
df.columns

#Represent features as vectors
vec_assembler = VectorAssembler(inputCols=['Recipientgender',
 'Stemcellsource',
 'Donorage',
 'Donorage35',
 'IIIV',
 'Gendermatch',
 'DonorABO',
 'Riskgroup',
 'Txpostrelapse',
 'Diseasegroup',
 'HLAmatch',
 'HLAmismatch',
 'HLAgrI',
 'Recipientage',
 'Recipientage10',
 'Recipientageint',
 'Relapse',
 'aGvHDIIIIV',
 'CD34kgx10d6',
 'ANCrecovery',
 'PLTrecovery',
 'time_to_aGvHD_III_IV',
 'survival_time'],outputCol='features')
features_df = vec_assembler.transform(df)
features_df.printSchema()

features_df.select('features').show(3,truncate=False)

#Use StandardScaler to initialize features and output them to features_ In the scaled column
from pyspark.ml.feature import StandardScaler
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")
scaled_df = standardScaler.fit(features_df).transform(features_df)
scaled_df.select("features", "features_scaled").show(1, truncate=False)

#Converts a character type survival_status to a numeric type survival_status_ The index value is 0,1
from pyspark.ml.feature import StringIndexer
survival_status_index = StringIndexer(inputCol="survival_status",outputCol="survival_status_index").fit(scaled_df)

#Check the conversion results. The M malignant value is 1 and the B benign value is 0
scaled_df = survival_status_index.transform(scaled_df)
model = scaled_df.select('survival_status','survival_status_index')
model.show(18)

root
 |-- id: integer (nullable = true)
 |-- Recipientgender: integer (nullable = true)
 |-- Stemcellsource: integer (nullable = true)
 |-- Donorage: double (nullable = true)
 |-- Donorage35: integer (nullable = true)
 |-- IIIV: integer (nullable = true)
 |-- Gendermatch: integer (nullable = true)
 |-- DonorABO: integer (nullable = true)
 |-- RecipientABO: double (nullable = true)
 |-- RecipientRh: double (nullable = true)
 |-- ABOmatch: double (nullable = true)
 |-- CMVstatus: double (nullable = true)
 |-- DonorCMV: double (nullable = true)
 |-- RecipientCMV: double (nullable = true)
 |-- Disease: string (nullable = true)
 |-- Riskgroup: integer (nullable = true)
 |-- Txpostrelapse: integer (nullable = true)
 |-- Diseasegroup: integer (nullable = true)
 |-- HLAmatch: integer (nullable = true)
 |-- HLAmismatch: integer (nullable = true)
 |-- Antigen: double (nullable = true)
 |-- Alel: double (nullable = true)
 |-- HLAgrI: integer (nullable = true)
 |-- Recipientage: double (nullable =

In [ ]:
train_df,test_df = scaled_df.randomSplit([0.8,0.2])

print((train_df.count(),len(train_df.columns)))
print((test_df.count(),len(test_df.columns)))

(121, 41)
(21, 41)


In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
log_reg = LogisticRegression().setLabelCol("survival_status_index").fit(train_df)

train_results = log_reg.evaluate(train_df).predictions
#The probability at index 0 is survival_status_index = 0. The probability at the first index is survival_status_index = 1 predicted
train_results.filter(train_results['survival_status_index']==1).filter(train_results['prediction']==1).select(['survival_status_index','prediction','probability']).show(24,False)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+---------------------+----------+-----------------------------+
|survival_status_index|prediction|probability                  |
+---------------------+----------+-----------------------------+
|1.0                  |1.0       |[1.8348772857275286E-218,1.0]|
|1.0                  |1.0       |[8.611424430535874E-115,1.0] |
|1.0                  |1.0       |[0.0,1.0]                    |
|1.0                  |1.0       |[6.214616544130525E-122,1.0] |
|1.0                  |1.0       |[8.404552589892978E-87,1.0]  |
|1.0                  |1.0       |[5.212555107494629E-191,1.0] |
|1.0                  |1.0       |[2.895481640678037E-72,1.0]  |
|1.0                  |1.0       |[3.360280848972171E-225,1.0] |
|1.0                  |1.0       |[1.295503059863138E-180,1.0] |
|1.0                  |1.0       |[1.4117206555165388E-93,1.0] |
|1.0                  |1.0       |[0.0,1.0]                    |
|1.0                  |1.0       |[0.0,1.0]                    |
|1.0                  |1.

In [ ]:
results = log_reg.evaluate(test_df).predictions
results.printSchema()

results.select(['survival_status_index','prediction']).show(24,False)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


root
 |-- id: integer (nullable = true)
 |-- Recipientgender: integer (nullable = true)
 |-- Stemcellsource: integer (nullable = true)
 |-- Donorage: double (nullable = true)
 |-- Donorage35: integer (nullable = true)
 |-- IIIV: integer (nullable = true)
 |-- Gendermatch: integer (nullable = true)
 |-- DonorABO: integer (nullable = true)
 |-- RecipientABO: double (nullable = true)
 |-- RecipientRh: double (nullable = true)
 |-- ABOmatch: double (nullable = true)
 |-- CMVstatus: double (nullable = true)
 |-- DonorCMV: double (nullable = true)
 |-- RecipientCMV: double (nullable = true)
 |-- Disease: string (nullable = true)
 |-- Riskgroup: integer (nullable = true)
 |-- Txpostrelapse: integer (nullable = true)
 |-- Diseasegroup: integer (nullable = true)
 |-- HLAmatch: integer (nullable = true)
 |-- HLAmismatch: integer (nullable = true)
 |-- Antigen: double (nullable = true)
 |-- Alel: double (nullable = true)
 |-- HLAgrI: integer (nullable = true)
 |-- Recipientage: double (nullable =

In [ ]:
true_postives = results[(results.survival_status_index == 1) & (results.prediction == 1)].count()
true_negatives = results[(results.survival_status_index == 0) & (results.prediction == 0)].count()
false_positives = results[(results.survival_status_index == 0) & (results.prediction == 1)].count()
false_negatives = results[(results.survival_status_index == 1) & (results.prediction == 0)].count()

#Accuracy
accuracy=float((true_postives+true_negatives) /(results.count()))
print("accuracy:",accuracy)

#recall 
recall = float(true_postives)/(true_postives + false_negatives)
print("recall:",recall)

#precision
precision = float(true_postives) / (true_postives + false_positives)
print("precision:",precision)

#F1 Score
F =  2 * (precision*recall) / (precision + recall)
print("F1 Score:",F)

accuracy: 0.9047619047619048
recall: 0.8333333333333334
precision: 0.8333333333333334
F1 Score: 0.8333333333333334


In [ ]:
#Support Vector Machine

In [ ]:
from pyspark.ml.classification import LinearSVC
svm_reg = LinearSVC().setLabelCol("survival_status_index").fit(train_df)

train_results = svm_reg.evaluate(train_df).predictions


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
results = svm_reg.evaluate(test_df).predictions
results.printSchema()

results.select(['survival_status_index','prediction']).show(24,False)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


root
 |-- id: integer (nullable = true)
 |-- Recipientgender: integer (nullable = true)
 |-- Stemcellsource: integer (nullable = true)
 |-- Donorage: double (nullable = true)
 |-- Donorage35: integer (nullable = true)
 |-- IIIV: integer (nullable = true)
 |-- Gendermatch: integer (nullable = true)
 |-- DonorABO: integer (nullable = true)
 |-- RecipientABO: double (nullable = true)
 |-- RecipientRh: double (nullable = true)
 |-- ABOmatch: double (nullable = true)
 |-- CMVstatus: double (nullable = true)
 |-- DonorCMV: double (nullable = true)
 |-- RecipientCMV: double (nullable = true)
 |-- Disease: string (nullable = true)
 |-- Riskgroup: integer (nullable = true)
 |-- Txpostrelapse: integer (nullable = true)
 |-- Diseasegroup: integer (nullable = true)
 |-- HLAmatch: integer (nullable = true)
 |-- HLAmismatch: integer (nullable = true)
 |-- Antigen: double (nullable = true)
 |-- Alel: double (nullable = true)
 |-- HLAgrI: integer (nullable = true)
 |-- Recipientage: double (nullable =

In [ ]:
true_postives = results[(results.survival_status_index == 1) & (results.prediction == 1)].count()
true_negatives = results[(results.survival_status_index == 0) & (results.prediction == 0)].count()
false_positives = results[(results.survival_status_index == 0) & (results.prediction == 1)].count()
false_negatives = results[(results.survival_status_index == 1) & (results.prediction == 0)].count()

#Accuracy
accuracy=float((true_postives+true_negatives) /(results.count()))
print("accuracy:",accuracy)

#recall 
recall = float(true_postives)/(true_postives + false_negatives)
print("recall:",recall)

#accuracy
precision = float(true_postives) / (true_postives + false_positives)
print("precision:",precision)

#F1 Score
F =  2 * (precision*recall) / (precision + recall)
print("F1-score:",F)

accuracy: 0.8571428571428571
recall: 0.6666666666666666
precision: 0.8
F1-score: 0.7272727272727272


In [ ]:
#Gratient-boosted Classıfıer

In [ ]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(labelCol="survival_status", featuresCol="features",maxIter=20)
gbt_model = gbt.fit(train_df)
gbt_prediction = gbt_model.transform(test_df)
gbt_prediction.select("prediction", "survival_status", "features").show()

+----------+---------------+--------------------+
|prediction|survival_status|            features|
+----------+---------------+--------------------+
|       0.0|              0|(23,[1,2,4,6,9,13...|
|       0.0|              0|[1.0,0.0,40.96986...|
|       0.0|              0|(23,[0,1,2,4,6,7,...|
|       1.0|              1|(23,[1,2,6,7,8,9,...|
|       0.0|              0|(23,[2,6,7,9,13,1...|
|       0.0|              0|[1.0,1.0,33.12054...|
|       0.0|              0|(23,[0,2,6,9,13,1...|
|       1.0|              0|[1.0,1.0,32.37260...|
|       1.0|              1|(23,[1,2,3,4,6,9,...|
|       1.0|              1|(23,[0,1,2,3,9,13...|
|       0.0|              0|[0.0,1.0,39.62739...|
|       0.0|              0|[1.0,1.0,40.39178...|
|       1.0|              1|[0.0,0.0,25.50684...|
|       0.0|              0|(23,[0,1,2,6,9,13...|
|       1.0|              1|[1.0,1.0,29.30684...|
|       1.0|              0|[1.0,1.0,30.42191...|
|       0.0|              0|[1.0,1.0,18.64657...|


In [ ]:
results = gbt_prediction
results.printSchema()

results.select(['survival_status_index','prediction']).show(24,False)

root
 |-- id: integer (nullable = true)
 |-- Recipientgender: integer (nullable = true)
 |-- Stemcellsource: integer (nullable = true)
 |-- Donorage: double (nullable = true)
 |-- Donorage35: integer (nullable = true)
 |-- IIIV: integer (nullable = true)
 |-- Gendermatch: integer (nullable = true)
 |-- DonorABO: integer (nullable = true)
 |-- RecipientABO: double (nullable = true)
 |-- RecipientRh: double (nullable = true)
 |-- ABOmatch: double (nullable = true)
 |-- CMVstatus: double (nullable = true)
 |-- DonorCMV: double (nullable = true)
 |-- RecipientCMV: double (nullable = true)
 |-- Disease: string (nullable = true)
 |-- Riskgroup: integer (nullable = true)
 |-- Txpostrelapse: integer (nullable = true)
 |-- Diseasegroup: integer (nullable = true)
 |-- HLAmatch: integer (nullable = true)
 |-- HLAmismatch: integer (nullable = true)
 |-- Antigen: double (nullable = true)
 |-- Alel: double (nullable = true)
 |-- HLAgrI: integer (nullable = true)
 |-- Recipientage: double (nullable =

In [ ]:
true_postives = results[(results.survival_status_index == 1) & (results.prediction == 1)].count()
true_negatives = results[(results.survival_status_index == 0) & (results.prediction == 0)].count()
false_positives = results[(results.survival_status_index == 0) & (results.prediction == 1)].count()
false_negatives = results[(results.survival_status_index == 1) & (results.prediction == 0)].count()

#Accuracy
accuracy=float((true_postives+true_negatives) /(results.count()))
print("accuracy:",accuracy)

#recall 
recall = float(true_postives)/(true_postives + false_negatives)
print("recall:",recall)

#accuracy
precision = float(true_postives) / (true_postives + false_positives)
print("precision:",precision)

#F1 Score
F =  2 * (precision*recall) / (precision + recall)
print("F1-score:",F)

accuracy: 0.9047619047619048
recall: 1.0
precision: 0.75
F1-score: 0.8571428571428571
